In [27]:
import pandas as pd
import numpy as np
import pickle
import math
from math import cos, sin, pi
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from rfpimp import permutation_importances

In [28]:
def dist_(x1,y1,x2,y2):
  try:
    x = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
  except:
    x = 0
  return x

### Exploring IBTrACS data - what cyclones have radii


In [29]:
df2 = pd.read_csv('IBTrACS_original.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (12,21,22,61,66,128,143,148) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [30]:
df2.head()

,SID,SEASON,NUMBER,BASIN,SUBBASIN,NAME,ISO_TIME,NATURE,LAT,LON,...,REU_GUSTP,USA_SEAHGT,USA_SEARAD,STORM_SPD,STORM_DR,year,month,day,hour,min
0,1842298N11080,1842,1,NI,BB,NOT_NAMED,1842-10-25 03:00:00,NR,10.90,80.30,...,NaN,NaN,NaN,9,266,1842,10,25,3,0
1,1842298N11080,1842,1,NI,BB,NOT_NAMED,1842-10-25 06:00:00,NR,10.87,79.83,...,NaN,NaN,NaN,9,267,1842,10,25,6,0
2,1842298N11080,1842,1,NI,BB,NOT_NAMED,1842-10-25 09:00:00,NR,10.84,79.35,...,NaN,NaN,NaN,9,267,1842,10,25,9,0
3,1842298N11080,1842,1,NI,BB,NOT_NAMED,1842-10-25 12:00:00,NR,10.82,78.88,...,NaN,NaN,NaN,9,267,1842,10,25,12,0
4,1842298N11080,1842,1,NI,BB,NOT_NAMED,1842-10-25 15:00:00,NR,10.80,78.40,...,NaN,NaN,NaN,9,268,1842,10,25,15,0


In [31]:
df2['TOK_R50_L']

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
669865   NaN
669866   NaN
669867   NaN
669868   NaN
669869   NaN
Name: TOK_R50_L, Length: 669870, dtype: float64

In [32]:
df2['LON_1'] = df2.groupby(['SID'])['LON'].shift(-1)
df2['LAT_1'] = df2.groupby(['SID'])['LAT'].shift(-1)

In [33]:
def big_wind(row):
  x1 = row['LAT']
  x2 = row['LAT_1']
  y1 = row['LON']
  y2 = row['LON_1']

  return dist_(x1,y1,x2,y2)
df2['dist'] = df2.apply(big_wind, axis=1)
df2['ISO_TIME'] = pd.to_datetime(df2['ISO_TIME'])
df2["rank"] = df2.groupby("SID")["ISO_TIME"].rank("dense", ascending=False)
df2["rank_1"] = df2.groupby("SID")["ISO_TIME"].rank("dense", ascending=True)

In [34]:
#df2['NEW_D_34'] = [df2['BOM_R34_NE']==df2['BOM_R34_NE']].NAME.unique()

In [35]:
for j in [34,50,64]:
  df2['max_'+str(j)] = df2[['USA_R'+str(j)+'_NE', 'USA_R'+str(j)+'_SE', 'USA_R'+str(j)+'_SW', 'USA_R'+str(j)+'_NW',
                            'REU_R'+str(j)+'_NE', 'REU_R'+str(j)+'_SE', 'REU_R'+str(j)+'_SW', 'REU_R'+str(j)+'_NW',
                            'BOM_R'+str(j)+'_NE', 'BOM_R'+str(j)+'_SE', 'BOM_R'+str(j)+'_SW', 'BOM_R'+str(j)+'_NW']].max(axis=1)  #USA_R34_NE

In [36]:
df2['max_50'] = df2[['max_50', 'TOK_R50_L']].max(axis=1)  #

In [37]:
df2['RMW'] = df2[['USA_RMW', 'REU_RMW', 'BOM_RMW']].max(axis=1)  #

In [38]:
df4 = df2[df2['max_34']==df2['max_34']][['NAME','SEASON']]

In [39]:
df2

,SID,SEASON,NUMBER,BASIN,SUBBASIN,NAME,ISO_TIME,NATURE,LAT,LON,...,min,LON_1,LAT_1,dist,rank,rank_1,max_34,max_50,max_64,RMW
0,1842298N11080,1842,1,NI,BB,NOT_NAMED,1842-10-25 03:00:00,NR,10.90,80.30,...,0,79.83,10.87,0.470956,64.0,1.0,NaN,NaN,NaN,NaN
1,1842298N11080,1842,1,NI,BB,NOT_NAMED,1842-10-25 06:00:00,NR,10.87,79.83,...,0,79.35,10.84,0.480937,63.0,2.0,NaN,NaN,NaN,NaN
2,1842298N11080,1842,1,NI,BB,NOT_NAMED,1842-10-25 09:00:00,NR,10.84,79.35,...,0,78.88,10.82,0.470425,62.0,3.0,NaN,NaN,NaN,NaN
3,1842298N11080,1842,1,NI,BB,NOT_NAMED,1842-10-25 12:00:00,NR,10.82,78.88,...,0,78.40,10.80,0.480416,61.0,4.0,NaN,NaN,NaN,NaN
4,1842298N11080,1842,1,NI,BB,NOT_NAMED,1842-10-25 15:00:00,NR,10.80,78.40,...,0,77.92,10.79,0.480104,60.0,5.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
669865,2019141N29291,2019,24,NaN,NaN,ANDREA,2019-05-20 21:00:00,NR,28.81,-68.74,...,0,-68.80,29.06,0.257099,5.0,2.0,69.0,NaN,NaN,66.0
669866,2019141N29291,2019,24,NaN,NaN,ANDREA,2019-05-21 00:00:00,NR,29.06,-68.80,...,0,-68.89,29.40,0.351710,4.0,3.0,69.0,NaN,NaN,63.0
669867,2019141N29291,2019,24,NaN,NaN,ANDREA,2019-05-21 03:00:00,NR,29.40,-68.89,...,0,-69.00,29.80,0.414849,3.0,4.0,69.0,NaN,NaN,60.0
669868,2019141N29291,2019,24,NaN,NaN,ANDREA,2019-05-21 06:00:00,NR,29.80,-69.00,...,0,-69.10,30.24,0.451221,2.0,5.0,69.0,NaN,NaN,58.0


In [40]:
df4 = df4.drop_duplicates()

In [41]:
len(df4)

1418

In [42]:
df4['in_tracks'] = True

### Estimating Radiouses of 34 50 60 kn winds 

In [43]:
df2['WMO_PRES_delta'] = df2.groupby(['SID'])['WMO_PRES'].diff(periods=-1)
df2.groupby('SID')['WMO_PRES_delta'].fillna(df2.median().iloc[0])

0         1969.0
1         1969.0
2         1969.0
3         1969.0
4         1969.0
           ...  
669865    1969.0
669866    1969.0
669867    1969.0
669868    1969.0
669869    1969.0
Name: WMO_PRES_delta, Length: 669870, dtype: float64

In [44]:
df2.groupby('SID')['dist'].fillna(df2.median().iloc[0])

0            0.470956
1            0.480937
2            0.470425
3            0.480416
4            0.480104
             ...     
669865       0.257099
669866       0.351710
669867       0.414849
669868       0.451221
669869    1969.000000
Name: dist, Length: 669870, dtype: float64

In [45]:
df2['dist'].fillna(df2.median().iloc[0])

0            0.470956
1            0.480937
2            0.470425
3            0.480416
4            0.480104
             ...     
669865       0.257099
669866       0.351710
669867       0.414849
669868       0.451221
669869    1969.000000
Name: dist, Length: 669870, dtype: float64

In [46]:
j#more 'BASIN','SUBBASIN' 'TRACK_TYPE' 'DIST2LAND','LANDFALL', 'STORM_DR'
df2[df2['max_50']==df2['max_50']]

,SID,SEASON,NUMBER,BASIN,SUBBASIN,NAME,ISO_TIME,NATURE,LAT,LON,...,LON_1,LAT_1,dist,rank,rank_1,max_34,max_50,max_64,RMW,WMO_PRES_delta
395872,1977161N06134,1977,44,WP,MM,RUTH,1977-06-15 06:00:00,TS,19.39,116.89,...,117.06,19.81,0.453100,50.0,43.0,NaN,20.0,NaN,NaN,NaN
395873,1977161N06134,1977,44,WP,MM,RUTH,1977-06-15 09:00:00,TS,19.81,117.06,...,117.23,20.25,0.471699,49.0,44.0,NaN,22.0,NaN,NaN,NaN
395874,1977161N06134,1977,44,WP,MM,RUTH,1977-06-15 12:00:00,TS,20.25,117.23,...,117.38,20.70,0.474342,48.0,45.0,NaN,25.0,NaN,NaN,NaN
395875,1977161N06134,1977,44,WP,MM,RUTH,1977-06-15 15:00:00,TS,20.70,117.38,...,117.52,21.17,0.490408,47.0,46.0,NaN,25.0,NaN,NaN,NaN
395876,1977161N06134,1977,44,WP,MM,RUTH,1977-06-15 18:00:00,TS,21.17,117.52,...,117.64,21.63,0.475395,46.0,47.0,NaN,25.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667974,2018307N14251,2018,98,EP,MM,XAVIER,2018-11-05 00:00:00,TS,18.27,-105.57,...,-105.68,18.32,0.120830,24.0,19.0,138.0,20.0,NaN,20.0,NaN
667975,2018307N14251,2018,98,EP,MM,XAVIER,2018-11-05 03:00:00,TS,18.32,-105.68,...,-105.80,18.33,0.120416,23.0,20.0,126.0,21.0,NaN,20.0,NaN
667976,2018307N14251,2018,98,EP,MM,XAVIER,2018-11-05 06:00:00,TS,18.33,-105.80,...,-105.88,18.36,0.085440,22.0,21.0,115.0,23.0,NaN,20.0,NaN
667977,2018307N14251,2018,98,EP,MM,XAVIER,2018-11-05 09:00:00,TS,18.36,-105.88,...,-106.00,18.43,0.138924,21.0,22.0,115.0,23.0,NaN,20.0,NaN


In [47]:
df2['BASIN'] = df2['BASIN'].fillna('NAM')

In [48]:
ib = df2[df2['max_50']==df2['max_50']][['NAME','SEASON', 'SID', 'WMO_PRES_delta', 'NUMBER',
                                      'BASIN','SUBBASIN','NAME','ISO_TIME','NATURE','LAT','LON','WMO_WIND','WMO_PRES',
                                        'TRACK_TYPE','DIST2LAND','LANDFALL',
                                       'STORM_SPD', 'dist',	'STORM_DR',	'year',	'month',
                                      'day', 'hour', 'min', 'max_34', 'max_50', 'max_64', 'rank', 'rank_1'
                                                                                                   
                                                                                                   
                                                                                                   ]]

In [49]:
#ib = pd.get_dummies(ib, columns=['BASIN','SUBBASIN', 'WMO_AGENCY','TRACK_TYPE',  'NATURE'])

In [50]:
ib

,NAME,SEASON,SID,WMO_PRES_delta,NUMBER,BASIN,SUBBASIN,NAME,ISO_TIME,NATURE,...,year,month,day,hour,min,max_34,max_50,max_64,rank,rank_1
395872,RUTH,1977,1977161N06134,NaN,44,WP,MM,RUTH,1977-06-15 06:00:00,TS,...,1977,6,15,6,0,NaN,20.0,NaN,50.0,43.0
395873,RUTH,1977,1977161N06134,NaN,44,WP,MM,RUTH,1977-06-15 09:00:00,TS,...,1977,6,15,9,0,NaN,22.0,NaN,49.0,44.0
395874,RUTH,1977,1977161N06134,NaN,44,WP,MM,RUTH,1977-06-15 12:00:00,TS,...,1977,6,15,12,0,NaN,25.0,NaN,48.0,45.0
395875,RUTH,1977,1977161N06134,NaN,44,WP,MM,RUTH,1977-06-15 15:00:00,TS,...,1977,6,15,15,0,NaN,25.0,NaN,47.0,46.0
395876,RUTH,1977,1977161N06134,NaN,44,WP,MM,RUTH,1977-06-15 18:00:00,TS,...,1977,6,15,18,0,NaN,25.0,NaN,46.0,47.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667974,XAVIER,2018,2018307N14251,NaN,98,EP,MM,XAVIER,2018-11-05 00:00:00,TS,...,2018,11,5,0,0,138.0,20.0,NaN,24.0,19.0
667975,XAVIER,2018,2018307N14251,NaN,98,EP,MM,XAVIER,2018-11-05 03:00:00,TS,...,2018,11,5,3,0,126.0,21.0,NaN,23.0,20.0
667976,XAVIER,2018,2018307N14251,NaN,98,EP,MM,XAVIER,2018-11-05 06:00:00,TS,...,2018,11,5,6,0,115.0,23.0,NaN,22.0,21.0
667977,XAVIER,2018,2018307N14251,NaN,98,EP,MM,XAVIER,2018-11-05 09:00:00,TS,...,2018,11,5,9,0,115.0,23.0,NaN,21.0,22.0


In [51]:
# Code modified by Raghuram on 6th April to fill the missing values with median
cols = ['WMO_WIND','WMO_PRES']
ib[cols] = ib.groupby('SID')[cols].ffill().fillna(ib[cols].median()).astype(int)

In [52]:
#ib['RMW_1'] = ib.groupby(['SID'])['RMW'].shift(-1)

In [53]:
# Code modified by Raghuram on 6th April to fill the missing values with median
cols_winds = ['max_34','max_50','max_64', 'dist']     
ib[cols] = ib.groupby('SID')[cols].ffill().fillna(ib[cols].median()).astype(int)

In [54]:
ib

,NAME,SEASON,SID,WMO_PRES_delta,NUMBER,BASIN,SUBBASIN,NAME,ISO_TIME,NATURE,...,year,month,day,hour,min,max_34,max_50,max_64,rank,rank_1
395872,RUTH,1977,1977161N06134,NaN,44,WP,MM,RUTH,1977-06-15 06:00:00,TS,...,1977,6,15,6,0,NaN,20.0,NaN,50.0,43.0
395873,RUTH,1977,1977161N06134,NaN,44,WP,MM,RUTH,1977-06-15 09:00:00,TS,...,1977,6,15,9,0,NaN,22.0,NaN,49.0,44.0
395874,RUTH,1977,1977161N06134,NaN,44,WP,MM,RUTH,1977-06-15 12:00:00,TS,...,1977,6,15,12,0,NaN,25.0,NaN,48.0,45.0
395875,RUTH,1977,1977161N06134,NaN,44,WP,MM,RUTH,1977-06-15 15:00:00,TS,...,1977,6,15,15,0,NaN,25.0,NaN,47.0,46.0
395876,RUTH,1977,1977161N06134,NaN,44,WP,MM,RUTH,1977-06-15 18:00:00,TS,...,1977,6,15,18,0,NaN,25.0,NaN,46.0,47.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667974,XAVIER,2018,2018307N14251,NaN,98,EP,MM,XAVIER,2018-11-05 00:00:00,TS,...,2018,11,5,0,0,138.0,20.0,NaN,24.0,19.0
667975,XAVIER,2018,2018307N14251,NaN,98,EP,MM,XAVIER,2018-11-05 03:00:00,TS,...,2018,11,5,3,0,126.0,21.0,NaN,23.0,20.0
667976,XAVIER,2018,2018307N14251,NaN,98,EP,MM,XAVIER,2018-11-05 06:00:00,TS,...,2018,11,5,6,0,115.0,23.0,NaN,22.0,21.0
667977,XAVIER,2018,2018307N14251,NaN,98,EP,MM,XAVIER,2018-11-05 09:00:00,TS,...,2018,11,5,9,0,115.0,23.0,NaN,21.0,22.0


In [55]:
df2['BASIN'].unique()

array(['NI', 'SI', 'NAM', 'EP', 'WP', 'SP', 'SA'], dtype=object)

In [56]:
ib.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45020 entries, 395872 to 667978
Data columns (total 30 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   NAME            45020 non-null  object        
 1   SEASON          45020 non-null  int64         
 2   SID             45020 non-null  object        
 3   WMO_PRES_delta  4808 non-null   float64       
 4   NUMBER          45020 non-null  int64         
 5   BASIN           45020 non-null  object        
 6   SUBBASIN        39822 non-null  object        
 7   NAME            45020 non-null  object        
 8   ISO_TIME        45020 non-null  datetime64[ns]
 9   NATURE          45020 non-null  object        
 10  LAT             45020 non-null  float64       
 11  LON             45020 non-null  float64       
 12  WMO_WIND        45020 non-null  int64         
 13  WMO_PRES        45020 non-null  int64         
 14  TRACK_TYPE      45020 non-null  object        
 

In [57]:
# Code modified by Raghuram on 6th April to fill the missing values with median
cols = ['WMO_PRES_delta']
ib[cols] = ib.groupby('SID')[cols].ffill().fillna(ib[cols].median()).astype(int)

In [58]:
ib = pd.get_dummies(ib, columns=['BASIN'])

In [59]:
ib = pd.get_dummies(ib, columns=['month']) 

In [60]:
ib['dist'] = ib['dist'].fillna(method = 'ffill')

In [61]:
import numpy as np
from sklearn.model_selection import train_test_split
column_names = ['LAT','LON','WMO_WIND','WMO_PRES','DIST2LAND','LANDFALL','STORM_SPD',	
                'STORM_DR', 'dist',	
                'month_1',	'month_2',	'month_3',	'month_4',	'month_5',	'month_6',
                'month_7',	'month_8',	'month_9',	'month_10',	'month_11',	'month_12',
                'rank', 'rank_1',
                'BASIN_EP','BASIN_NI','BASIN_WP']
X = ib[column_names]
y = ib['max_50']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)



In [62]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
#model.fit(X, y)
# Code modified by Raghuram on 6th April to avoid data leakage
model.fit(X_train, y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [63]:
y_pred = model.predict(X_test)

In [64]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 23.626157834938653
Mean Squared Error: 8416.811068585192
Root Mean Squared Error: 91.74317995679674


In [65]:
df66 = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df68 = df66.head(25)

In [66]:
df66

,Actual,Predicted
661606,100.0,101.148439
598086,70.0,60.878056
654830,99.0,65.746217
445233,125.0,63.178425
589263,120.0,57.532002
...,...,...
615352,35.0,36.839222
569986,100.0,108.481990
590002,30.0,81.059593
591169,40.0,32.449086


### Model - Random Forest Regressor

In [67]:
# Code for 'Model - Random Forest Regressor' added by Raghuram on 6th April 2020
from sklearn.ensemble import RandomForestRegressor

model2 = RandomForestRegressor(n_estimators = 60, random_state = 0)
model2.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=60, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [68]:
y_pred = model2.predict(X_test)

In [69]:
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 11.110737923762088
Mean Squared Error: 3510.2413874492386
Root Mean Squared Error: 59.24729012747535


**Observation**: With this RandomForestRegressor model, MAE reduced from 51 to 20.9 and RMSE reduced from 215 to 104 when compared with Linear Regression model.

In [70]:
df_rf = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df_rf.head(125)

,Actual,Predicted
661606,100.0,103.000000
598086,70.0,62.400000
654830,99.0,87.433333
445233,125.0,121.783333
589263,120.0,81.200000
...,...,...
635392,30.0,29.166667
625856,40.0,50.866667
602067,70.0,69.150000
527699,80.0,79.200000


In [71]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 800, num = 22)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 22)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [50, 85, 121, 157, 192, 228, 264, 300, 335, 371, 407, 442, 478, 514, 550, 585, 621, 657, 692, 728, 764, 800], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 14, 19, 24, 29, 33, 38, 43, 48, 52, 57, 62, 67, 71, 76, 81, 86, 90, 95, 100, 105, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [72]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random_2 = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random_2.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   46.9s
/opt/conda/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed: 12.9min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 28.6min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                              

In [73]:
rf_random_2.best_params_

{'n_estimators': 764,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 100,
 'bootstrap': False}

In [74]:
y_pred_2 = rf_random_2.predict(X_test)

In [75]:
df_rf.describe()

,Actual,Predicted
count,14857.000000,14857.000000
mean,72.213704,72.379825
std,95.061051,59.374998
min,5.000000,10.733333
25%,40.000000,45.850000
50%,60.000000,64.800000
75%,90.000000,90.750000
max,5399.000000,3858.683333


In [76]:
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 11.110737923762088
Mean Squared Error: 3510.2413874492386
Root Mean Squared Error: 59.24729012747535


In [77]:
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred_2))  
print('Mean Squared Error:', mean_squared_error(y_test, y_pred_2))  
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred_2)))

Mean Absolute Error: 9.102373679103204
Mean Squared Error: 3340.0316032497944
Root Mean Squared Error: 57.79300652544211


##Populating with predictions 

In [95]:
filename = 'rf_random_50.sav'
pickle.dump(rf_random_2, open(filename, 'wb'))
 



In [96]:
# load the model from disk
rf_random_50 = pickle.load(open('rf_random_50.sav', 'rb'))
result = rf_random_50.score(X_test, y_test)
print(result)

0.6303637248347166


In [97]:
pip install rfpimp

Note: you may need to restart the kernel to use updated packages.


In [98]:
perm_imp_rfpimp

NameError: name 'perm_imp_rfpimp' is not defined

In [ ]:
rf_random_50

Unwraping no names

In [ ]:
df[df['in_ibtrax'] != True][df['in_pre'] == True]

In [ ]:
pre_df  = pd.read_csv('OUTPUT_WBI_cyclones.csv', sep = ';', low_memory=False)

In [ ]:
pre_df_unique  = pre_df[['NAME',	'Year']]

In [ ]:
pre_df_unique = pre_df_unique.drop_duplicates()

In [ ]:
pre_df['in_pre'] = True

In [ ]:
df2['in_ibtrax'] = True

In [ ]:
df = pd.merge(df2, pre_df, how='outer', left_on = ['NAME', 'SEASON'], right_on=['NAME','Year'], suffixes=('', '_pre'))

In [ ]:
df = pd.get_dummies(df, columns=['BASIN','month'])

In [ ]:
#y = df['max_50'][df['max_50']!=df['max_50']]

In [ ]:
df['WMO_WIND'] =df.groupby('SID')['WMO_WIND'].fillna(df['WMO_WIND'].median())
df['WMO_WIND'] = df['WMO_WIND'].fillna(df['WMO_WIND'].median())
df['WMO_PRES']=df.groupby('SID')['WMO_PRES'].fillna(df['WMO_PRES'].median())
df['WMO_PRES'] = df['WMO_PRES'].fillna(df['WMO_PRES'].median())
df['STORM_SPD']=df.groupby('SID')['STORM_SPD'].fillna(df['STORM_SPD'].median())
df['STORM_SPD'] = df['STORM_SPD'].fillna(df['STORM_SPD'].median())
df['STORM_SPD'] = df['STORM_SPD'].fillna(df['STORM_SPD'].median())
df['STORM_DR']=df.groupby('SID')['STORM_DR'].fillna(df['STORM_DR'].median())
df['STORM_DR']=df.groupby('SID')['STORM_DR'].fillna(df['STORM_DR'].median())

In [ ]:
df['dist'] = df['dist'].fillna(method='ffill')

##TESTING ACCURACY ON NEW DATA

In [ ]:

import numpy as np
from sklearn.model_selection import train_test_split
column_names = ['LAT','LON','WMO_WIND','WMO_PRES','DIST2LAND','LANDFALL','STORM_SPD',	
                'STORM_DR', 'dist',	
                'month_1.0',	'month_2.0',	'month_3.0',	'month_4.0',	'month_5.0',	'month_6.0',
                'month_7.0',	'month_8.0',	'month_9.0',	'month_10.0',	'month_11.0',	'month_12.0',
                'rank', 'rank_1',
                'BASIN_EP','BASIN_NI','BASIN_WP']
X = df[column_names][df['max_50']==df['max_50']]
y = df['max_50'][df['max_50']==df['max_50']]

In [ ]:
df['dist'] = df['dist'].fillna(method='ffill')

In [ ]:
X

In [ ]:
y_pred_50 = rf_random_50.predict(X)

In [ ]:
df_rf_50 = pd.DataFrame({'Actual': y, 'Predicted': y_pred_50})

In [ ]:
print('Mean Absolute Error:', mean_absolute_error(y, y_pred_50))  
print('Mean Squared Error:', mean_squared_error(y, y_pred_50))  
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y, y_pred_50)))

##PREDICTIONS

In [ ]:
df_to_pred = df[df['max_50']!=df['max_50']][df['SID']==df['SID']]

In [160]:
df_to_pred['dist'] = df_to_pred['dist'].fillna(method = 'ffill')

In [161]:
df['BASIN_NI']

0         1
1         1
2         1
3         1
4         1
         ..
693177    0
693178    0
693179    0
693180    0
693181    0
Name: BASIN_NI, Length: 693182, dtype: uint8

In [ ]:

import numpy as np
from sklearn.model_selection import train_test_split
column_names = ['LAT','LON','WMO_WIND','WMO_PRES','DIST2LAND','LANDFALL','STORM_SPD',	
                'STORM_DR', 'dist',	
                'month_1.0',	'month_2.0',	'month_3.0',	'month_4.0',	'month_5.0',	'month_6.0',
                'month_7.0',	'month_8.0',	'month_9.0',	'month_10.0',	'month_11.0',	'month_12.0',
                'rank', 'rank_1',
                'BASIN_EP','BASIN_NI','BASIN_WP']
X = df[column_names][df['in_pre'] == True]
y = df['max_50'][df['in_pre'] == True]


In [ ]:
y_pred_50 = rf_random_50.predict(X)

In [ ]:
y_pred_50

In [ ]:
X.loc[:,'Outcome'] = y_pred_50
X.to_csv('max_50_.csv')
len(X)